# Template Network Definitions

<i>Version 2</i>

## References

1. ["Maintaining Knowledge about Temporal Intervals" by James F. Allen](https://cse.unl.edu/~choueiry/Documents/Allen-CACM1983.pdf) - Allen's original paper (PDF)
1. [Allen's Interval Algebra](https://www.ics.uci.edu/~alspaugh/cls/shr/allen.html) or [here](https://thomasalspaugh.org/pub/fnd/allen.html) - summarizes Allen's algebra of proper time intervals
1. [W3C Time Ontology in OWL](https://www.w3.org/TR/owl-time/) - temporal vocabulary used here is based on the W3C vocabulary of time
1. [bitsets Python package](https://bitsets.readthedocs.io/en/stable/) - used to implement Algebra relation sets and operations
1. [NetworkX Python package](http://networkx.github.io/) - used to represent directed graph of constraints
1. [Python format string syntax](https://docs.python.org/3/library/string.html#format-string-syntax) - used in Algebra summary method
1. [Spatial Ontology](https://www.w3.org/2017/sdwig/bp/) - I'm still looking for a standard spatial vocabulary; maybe start here
1. [Qualitative Spatial Relations (QSR) Library](https://qsrlib.readthedocs.io/en/latest/index.html) - an alternative library to the one defined here

## Dependencies

In [8]:
import os
import qualreas as qr
import numpy as np

In [9]:
path = os.path.join(os.getenv('PYPROJ'), 'qualreas')

In [10]:
#pt_alg = qr.Algebra(os.path.join(path, "Algebras/LinearPointAlgebra.json"))
#pt_alg = qr.Algebra(os.path.join(path, "Algebras/RightBranchingPointAlgebra.json"))
pt_alg = qr.Algebra(os.path.join(path, "Algebras/LeftBranchingPointAlgebra.json"))

## Create Algebra Elements using 4-Point Networks

### Define 4-Point Network Generator

In [11]:
def four_point_network(alg, lessthan_symbol, startname="StartPt", endname="EndPt",
                       verbose=False):
    '''Create four Temporal Entities that represent time points and use them
    to express two independent intervals. For example, (s1,e1) and (s2,e2),
    where s1 < e1 and s2 < e2, represents two proper intervals.  Using '<|='
    instead of '<', would represent two intervals where one or both might
    be points.  Return the network and the four temporal entities.'''
    net = qr.Network(alg, "Four Point Network")
    lessthan = alg.relset(lessthan_symbol)
    # Start & End Points of Interval 1
    start1 = qr.TemporalEntity(["Point"], name=startname+"1")
    end1   = qr.TemporalEntity(["Point"], name=endname+"1")
    # Start & End Points of Interval 2
    start2 = qr.TemporalEntity(["Point"], name=startname+"2")
    end2   = qr.TemporalEntity(["Point"], name=endname+"2")
    net.add_constraint(start1, end1, lessthan, verbose)
    net.add_constraint(start2, end2, lessthan, verbose)
    entities = [start1, end1, start2, end2]
    return net, entities

def constraint_matrix_to_list(net, entities):
    if not entities:
        entities = net.nodes
    result = []
    for a in entities:
        row = []
        for b in entities:
            row.append(str(net.edges[a, b]['constraint']))
        result.append(row)
    return result

# Map 4-Point Network "Signatures" to Typical Names
key_name_mapping = {
    '<,<,<,<': 'B', '>,>,>,>': 'BI',
    '>,<,>,<': 'D', '<,<,>,>': 'DI',
    '=,<,>,=': 'E', '=,=,=,=': 'PE',
    '>,<,>,=': 'F', '<,<,>,=': 'FI',
    '<,<,=,<': 'M', '>,=,>,>': 'MI',
    '<,<,>,<': 'O', '>,<,>,>': 'OI',
    '=,<,>,<': 'S', '=,<,>,>': 'SI',
    '>,=,>,=': 'PF', '<,<,=,=': 'PFI',
    '=,<,=,<': 'PS', '=,=,>,>': 'PSI',
    '<,<,>,r~': 'RO', '<,<,r~,r~': 'RB',
    '=,<,>,r~': 'RS', '>,<,>,r~': 'ROI',
    '>,r~,>,r~': 'RBI', 'r~,r~,r~,r~': 'R~',
    'l~,<,>,<': 'LO', 'l~,<,>,=': 'LF',
    'l~,<,>,>': 'LOI', 'l~,l~,>,>': 'LBI',
    'l~,<,l~,<': 'LB', 'l~,l~,l~,l~': 'L~'
}

# Viewing the network as a matrix, 'elem13', below, refers to the element in row 1 col 3,
# and so on for 'elem23', etc.  The matrix is 4x4, so if we partition it into four 2x2
# matrices, then the two partiions on the diagonal represent two intervals and the two
# off-diagonal partitions represent how those two intervals relate to each other.
# Also, the off-diagonal 2x2 partitions are converse transposes of each other.
# Oh, and the intervals represented by the diagonal partitions could be intervals,
# proper intervals, or points.

def generate_consistent_networks(alg, lessthan="<", startname="StartPt", endname="EndPt",
                                 verbose=False):
    consistent_nets = dict()
    for elem13 in alg.elements:
        for elem23 in alg.elements:
            for elem14 in alg.elements:
                for elem24 in alg.elements:
                    net, pts = four_point_network(alg, lessthan, startname, endname)
                    pt1, pt2, pt3, pt4 = pts
                    rs13 = alg.relset(elem13)
                    rs23 = alg.relset(elem23)
                    rs14 = alg.relset(elem14)
                    rs24 = alg.relset(elem24)
                    net.add_constraint(pt1, pt3, rs13)
                    net.add_constraint(pt2, pt3, rs23)
                    net.add_constraint(pt1, pt4, rs14)
                    net.add_constraint(pt2, pt4, rs24)
                    if net.propagate():
                        elem_key = ",".join([str(rs13), str(rs14), str(rs23), str(rs24)])
                        consistent_nets[key_name_mapping[elem_key]] = net
                        if verbose:
                            print("==========================")
                            if elem_key in key_name_mapping:
                                print(key_name_mapping[elem_key])
                            else:
                                print("UNKNOWN")
                            print(np.matrix(constraint_matrix_to_list(net, pts)))
    print(f"\n{len(consistent_nets)} consistent networks")
    return consistent_nets

### Generating a 4-Point Network that Represents 2 Intervals

In [29]:
net4pt, pts = four_point_network(pt_alg, "<|=")
net4pt.summary()


Four Point Network: 4 nodes, 8 edges
  Algebra: LeftBranchingPointAlgebra
  StartPt1:
    => StartPt1: =
    => EndPt1: <|=
  EndPt1:
    => EndPt1: =
    => StartPt1: =|>
  StartPt2:
    => StartPt2: =
    => EndPt2: <|=
  EndPt2:
    => EndPt2: =
    => StartPt2: =|>


In [30]:
for pt in pts:
    print(pt)

<TemporalEntity StartPt1 ['Point']>
<TemporalEntity EndPt1 ['Point']>
<TemporalEntity StartPt2 ['Point']>
<TemporalEntity EndPt2 ['Point']>


## Derive Algebra Elements

A 4-point network, like that generated above, only has constraints specified so that the first two points define an interval, and same for the second two points.  No constraints are specified between the two implied intervals (e.g., no constraint between StartPt1/EndPt1 and StartPt2/EndPt2).  Depending on which point algebra is used there are either 3^4 (81) or 4^4 (256) different ways the unassigned constraint pairs can be made.  The function,  <i>generate_consistent_networks</i> tries all of these possibilities and returns the ones that are consistent.  Doing this for the linear point algebra ('<', '=', '>') results in 13 consistent networks that correspond to Allen's Temporal Algebra of Proper Time Intervals.  Using ('<|=', '=', '>|=") results in 18 consistent networks that are a superset of Allen's relations that includes 5 additional relations that integrate points into the algebra.  Using ('<|=', '=', '>|=', '~'), where '~' is either the left-incomparable or right-incomparable relation of the left- or right-branching time point algebra will result in 24 consistent networks that integrate points into a left- or right-branching time interval algebra.

In [16]:
consistent_nets = generate_consistent_networks(pt_alg, lessthan="<|=", verbose=True)

B
[['=' '<|=' '<' '<']
 ['=|>' '=' '<' '<']
 ['>' '>' '=' '<|=']
 ['>' '>' '=|>' '=']]
M
[['=' '<' '<' '<']
 ['>' '=' '=' '<']
 ['>' '=' '=' '<']
 ['>' '>' '>' '=']]
PFI
[['=' '<' '<' '<']
 ['>' '=' '=' '=']
 ['>' '=' '=' '=']
 ['>' '=' '=' '=']]
O
[['=' '<' '<' '<']
 ['>' '=' '>' '<']
 ['>' '<' '=' '<']
 ['>' '>' '>' '=']]
FI
[['=' '<' '<' '<']
 ['>' '=' '>' '=']
 ['>' '<' '=' '<']
 ['>' '=' '>' '=']]
DI
[['=' '<' '<' '<']
 ['>' '=' '>' '>']
 ['>' '<' '=' '<|=']
 ['>' '<' '=|>' '=']]
PS
[['=' '=' '=' '<']
 ['=' '=' '=' '<']
 ['=' '=' '=' '<']
 ['>' '>' '>' '=']]
PE
[['=' '=' '=' '=']
 ['=' '=' '=' '=']
 ['=' '=' '=' '=']
 ['=' '=' '=' '=']]
S
[['=' '<' '=' '<']
 ['>' '=' '>' '<']
 ['=' '<' '=' '<']
 ['>' '>' '>' '=']]
E
[['=' '<' '=' '<']
 ['>' '=' '>' '=']
 ['=' '<' '=' '<']
 ['>' '=' '>' '=']]
SI
[['=' '<' '=' '<']
 ['>' '=' '>' '>']
 ['=' '<' '=' '<']
 ['>' '<' '>' '=']]
PSI
[['=' '<' '=' '=']
 ['>' '=' '>' '>']
 ['=' '<' '=' '=']
 ['=' '<' '=' '=']]
D
[['=' '<|=' '>' '<']
 ['=|>' 

In [31]:
consistent_nets

{'B': <qualreas.Network at 0x7ffac07d3d68>,
 'M': <qualreas.Network at 0x7ffa90bf3ac8>,
 'PFI': <qualreas.Network at 0x7ffac07d3ef0>,
 'O': <qualreas.Network at 0x7ffac07dbfd0>,
 'FI': <qualreas.Network at 0x7ffac07d3f28>,
 'DI': <qualreas.Network at 0x7ffac07dbdd8>,
 'PS': <qualreas.Network at 0x7ffac07ecfd0>,
 'PE': <qualreas.Network at 0x7ffac07e35c0>,
 'S': <qualreas.Network at 0x7ffac07d3d30>,
 'E': <qualreas.Network at 0x7ffac07e3ba8>,
 'SI': <qualreas.Network at 0x7ffa90c28e10>,
 'PSI': <qualreas.Network at 0x7ffa90525cc0>,
 'D': <qualreas.Network at 0x7ffac07d3cc0>,
 'F': <qualreas.Network at 0x7ffac07edd30>,
 'OI': <qualreas.Network at 0x7ffac07edcf8>,
 'PF': <qualreas.Network at 0x7ffac07f0a90>,
 'MI': <qualreas.Network at 0x7ffac07f08d0>,
 'BI': <qualreas.Network at 0x7ffac07f0f98>,
 'LO': <qualreas.Network at 0x7ffac07f0dd8>,
 'LF': <qualreas.Network at 0x7ffac07d3c88>,
 'LOI': <qualreas.Network at 0x7ffac07f0470>,
 'LBI': <qualreas.Network at 0x7ffac0807550>,
 'LB': <qualr

In [17]:
consistent_nets['B'].get_entity_by_name('StartPt2')

<TemporalEntity StartPt2 ['Point']>

In [18]:
consistent_nets['B'].get_entity_by_name('foo')

In [19]:
orig = consistent_nets['B']

In [20]:
orig.print_as_matrix()

['StartPt1', 'EndPt1', 'StartPt2', 'EndPt2']
  =  <|=  <  <
  =|>  =  <  <
  >  >  =  <|=
  >  >  =|>  =


In [21]:
orig_nodes = list(orig.nodes())
orig_nodes

[<TemporalEntity StartPt1 ['Point']>,
 <TemporalEntity EndPt1 ['Point']>,
 <TemporalEntity StartPt2 ['Point']>,
 <TemporalEntity EndPt2 ['Point']>]

In [22]:
orig_nodes[0]

<TemporalEntity StartPt1 ['Point']>

In [23]:
orig_nodes[1]

<TemporalEntity EndPt1 ['Point']>

In [24]:
for edge in orig.edges:
    print(edge)

(<TemporalEntity StartPt1 ['Point']>, <TemporalEntity StartPt1 ['Point']>)
(<TemporalEntity StartPt1 ['Point']>, <TemporalEntity EndPt1 ['Point']>)
(<TemporalEntity StartPt1 ['Point']>, <TemporalEntity StartPt2 ['Point']>)
(<TemporalEntity StartPt1 ['Point']>, <TemporalEntity EndPt2 ['Point']>)
(<TemporalEntity EndPt1 ['Point']>, <TemporalEntity EndPt1 ['Point']>)
(<TemporalEntity EndPt1 ['Point']>, <TemporalEntity StartPt1 ['Point']>)
(<TemporalEntity EndPt1 ['Point']>, <TemporalEntity StartPt2 ['Point']>)
(<TemporalEntity EndPt1 ['Point']>, <TemporalEntity EndPt2 ['Point']>)
(<TemporalEntity StartPt2 ['Point']>, <TemporalEntity StartPt2 ['Point']>)
(<TemporalEntity StartPt2 ['Point']>, <TemporalEntity EndPt2 ['Point']>)
(<TemporalEntity StartPt2 ['Point']>, <TemporalEntity StartPt1 ['Point']>)
(<TemporalEntity StartPt2 ['Point']>, <TemporalEntity EndPt1 ['Point']>)
(<TemporalEntity EndPt2 ['Point']>, <TemporalEntity EndPt2 ['Point']>)
(<TemporalEntity EndPt2 ['Point']>, <TemporalEnti

In [25]:
orig.edges[orig_nodes[0], orig_nodes[1]]['constraint']

relset(['<', '='])

In [26]:
s1 = consistent_nets['B'].get_entity_by_name('StartPt1')
e1 = consistent_nets['B'].get_entity_by_name('EndPt1')
consistent_nets['B'].edges[s1, e1]['constraint']

relset(['<', '='])

In [27]:
consistent_nets['B'].print_as_matrix()

['StartPt1', 'EndPt1', 'StartPt2', 'EndPt2']
  =  <|=  <  <
  =|>  =  <  <
  >  >  =  <|=
  >  >  =|>  =


In [28]:
consistent_nets['B'].print_as_matrix(['StartPt2', 'EndPt2', 'StartPt1', 'EndPt1'])

['StartPt2', 'EndPt2', 'StartPt1', 'EndPt1']
  =  <|=  >  >
  =|>  =  >  >
  <  <  =  <|=
  <  <  =|>  =
